<a href="https://colab.research.google.com/github/fillobissi/geoinformatics_project/blob/main/250417_Pixelwise_threshods_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Pixelwise threshods extraction**

In [ ]:
!pip install dask netCDF4 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.6 MB/s eta 0:00:00


In [ ]:
!pip install dask[complete] --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.0 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

**01 Estrazione del valore massimo di temperatura per ciascun giorno e per ciascun pixel**

Ottengo 15705 file che corrispondono ai giorni del mio datset

Tempo stimato 41 minuti

In [ ]:
# === CONFIG ===
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/max_temperature_era5-81_23.nc"
OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/hw_max_daily"
VAR_NAME = "TMAX_2M"
CHUNK_DICT = {"time": 365, "rlat": 50, "rlon": 50}
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === UTILS ===
def format_duration(seconds):
    minutes = int(seconds // 60)
    sec = int(seconds % 60)
    return f"{minutes}m {sec}s"

def print_progress(day, total_days, start_time, end_time):
    percent = 100 * day / total_days
    duration = (end_time - start_time).total_seconds()
    formatted_duration = format_duration(duration)
    print(f"[Progress] Day {day}/{total_days} ({percent:.1f}%) | "
          f"Start: {start_time.strftime('%H:%M:%S')} | End: {end_time.strftime('%H:%M:%S')} | "
          f"Duration: {formatted_duration}")

# === LOAD DATASET ===
print("Loading dataset and applying chunking...")
ds = xr.open_dataset(DATASET_PATH).chunk(CHUNK_DICT)
ds = ds.assign_coords(date=ds.time.dt.floor('D'))

# === GIORNI UNICI ===
unique_days = pd.to_datetime(np.unique(ds["date"].values)).sort_values()
total_days = len(unique_days)

# === VERIFICA FILE GIÀ SALVATI
already_done = {f for f in os.listdir(OUTPUT_DIR) if f.endswith(".nc")}
already_done_days = {pd.to_datetime(f.replace("max_day_", "").replace(".nc", "")) for f in already_done}
print(f"Found {len(already_done_days)} days already computed. Resuming from the next one...")

# === MAIN LOOP ===
for i, day in enumerate(unique_days):
    if day in already_done_days:
        continue

    start_time = datetime.now()
    sel = ds.sel(time=ds.date == np.datetime64(day))
    max_day = sel[VAR_NAME].max(dim="time")

    # Salva risultato
    output_path = os.path.join(OUTPUT_DIR, f"max_day_{day.strftime('%Y%m%d')}.nc")
    max_day.name = f"TMAX_daily"
    max_day.to_dataset().to_netcdf(output_path)

    end_time = datetime.now()
    print_progress(i + 1, total_days, start_time, end_time)

Output streaming troncato alle ultime 5000 righe.
[Progress] Day 10706/15705 (68.2%) | Start: 16:20:20 | End: 16:20:21 | Duration: 0m 0s
[Progress] Day 10707/15705 (68.2%) | Start: 16:20:21 | End: 16:20:21 | Duration: 0m 0s
[Progress] Day 10708/15705 (68.2%) | Start: 16:20:21 | End: 16:20:21 | Duration: 0m 0s
[Progress] Day 10709/15705 (68.2%) | Start: 16:20:21 | End: 16:20:21 | Duration: 0m 0s
[Progress] Day 10710/15705 (68.2%) | Start: 16:20:21 | End: 16:20:22 | Duration: 0m 0s
[Progress] Day 10711/15705 (68.2%) | Start: 16:20:22 | End: 16:20:22 | Duration: 0m 0s
[Progress] Day 10712/15705 (68.2%) | Start: 16:20:22 | End: 16:20:22 | Duration: 0m 0s
[Progress] Day 10713/15705 (68.2%) | Start: 16:20:22 | End: 16:20:22 | Duration: 0m 0s
[Progress] Day 10714/15705 (68.2%) | Start: 16:20:22 | End: 16:20:22 | Duration: 0m 0s
[Progress] Day 10715/15705 (68.2%) | Start: 16:20:22 | End: 16:20:22 | Duration: 0m 0s
[Progress] Day 10716/15705 (68.2%) | Start: 16:20:22 | End: 16:20:23 | Duration:

**02 Unisco i massimi di ciascun giorno in un unico file 3d (lat - lon - 15705 date)**

Tempo stimato 37 minuti
Dimensione output cube stimata 2gb

PRIMA DI LANCIARE IL CODICE SUCCESSIVO ELIMINARE DAL DRIVE IL DATASET COMPLETO RAW PER RAGIONI DI SPAZIO

In [3]:
# === PATH CONFIGURATION ===
INPUT_DIR = "/content/drive/MyDrive"
OUTPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/tmax_daily_cube.nc"
VAR_NAME = "TMAX_daily"

# Elimina file se esiste già
try:
    os.remove(OUTPUT_FILE)
    print("Vecchio file rimosso.")
except FileNotFoundError:
    print("Nessun file precedente trovato.")


# === LIST FILES ===
files = sorted([
    f for f in os.listdir(INPUT_DIR)
    if f.startswith("max_day_") and f.endswith(".nc")
])

print(f"Found {len(files)} files to concatenate...")

# === PREPARE DATASETS ===
datasets = []
times = []

for f in files:
    full_path = os.path.join(INPUT_DIR, f)

    # Estrai la data dal nome del file
    date_str = f.replace("max_day_", "").replace(".nc", "")
    time = pd.to_datetime(date_str, format="%Y%m%d")
    times.append(time)

    ds = xr.open_dataset(full_path)

    # Aggiungi coordinata temporale
    ds = ds.expand_dims({"time": [time]})
    datasets.append(ds)

# === CONCATENA E SALVA ===
combined = xr.concat(datasets, dim="time")
combined = combined.sortby("time")
combined.to_netcdf(OUTPUT_FILE)

print(f"Saved full data cube to: {OUTPUT_FILE}")
print(f"Removing original .nc files...")

# === DELETE ORIGINAL FILES ===
for f in files:
    os.remove(os.path.join(INPUT_DIR, f))

print("All done. Single-day files deleted.")

Nessun file precedente trovato.
Found 15705 files to concatenate...
Saved full data cube to: /content/drive/MyDrive/Colab Notebooks/tmax_daily_cube.nc
Removing original .nc files...
All done. Single-day files deleted.


Verifica dell'integrità del data cube generato nel codice sovrastante

In [4]:
import xarray as xr
import numpy as np
import pandas as pd

# === PATH AL CUBO DI MASSIMI GIORNALIERI ===
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/tmax_daily_cube.nc"
VAR_NAME = "TMAX_daily"  # nome variabile interno

# === CARICA IL DATASET ===
ds = xr.open_dataset(DATASET_PATH)
data = ds[VAR_NAME]

# === DIMENSIONI PRINCIPALI ===
print("✅ Dimensioni del dataset:")
print(data.dims)

# === CHECK 1: Numero totale di giorni
n_giorni = data.sizes["time"]
print(f"\n🗓️  Numero totale di giorni nel dataset: {n_giorni}")
assert n_giorni == 15705, "⚠️ Il numero di giorni non è 15705! Potrebbero mancare dei file."

# === CHECK 2: Date effettive
if "time" in data.coords:
    date_index = pd.to_datetime(data["time"].values)
    print(f"📅 Primo giorno: {date_index[0].date()}, Ultimo giorno: {date_index[-1].date()}")

    # Verifica che le date siano tutte consecutive e senza buchi
    diff = pd.Series(date_index).diff().dropna()
    if (diff != pd.Timedelta(days=1)).any():
        print("❌ ATTENZIONE: Ci sono giorni mancanti nella sequenza temporale!")
    else:
        print("✅ Tutti i giorni sono consecutivi e senza buchi.")
else:
    print("⚠️ Coordinata 'time' non trovata nel dataset.")

# === CHECK 3: Esistono giorni con TUTTI valori NaN?
nan_days = data.isnull().all(dim=["rlat", "rlon"]).sum().item()
print(f"🔍 Giorni con tutti i valori NaN: {nan_days}")

if nan_days > 0:
    print("⚠️ ATTENZIONE: alcuni giorni non contengono alcun valore valido!")
else:
    print("✅ Tutti i giorni contengono almeno un valore valido.")

# === CHECK 4: Valori min/max generali
min_val = float(data.min().values)
max_val = float(data.max().values)
print(f"\n🌡️ Valori globali nel cubo: Min = {min_val:.2f} °C, Max = {max_val:.2f} °C")


✅ Dimensioni del dataset:
('time', 'rlat', 'rlon')

🗓️  Numero totale di giorni nel dataset: 15705
📅 Primo giorno: 1981-01-01, Ultimo giorno: 2023-12-31
✅ Tutti i giorni sono consecutivi e senza buchi.
🔍 Giorni con tutti i valori NaN: 0
✅ Tutti i giorni contengono almeno un valore valido.

🌡️ Valori globali nel cubo: Min = 240.33 °C, Max = 320.18 °C


**03 Creo le soglie per ciascun pixel per ciascun giorno dell'anno (lat - lon - 366 date)**

In [5]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from datetime import datetime

# === CONFIG ===
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/tmax_daily_cube.nc"
OUTPUT_FILE = "/content/drive/MyDrive/Colab Notebooks/thresholds_15day_90p.nc"
VAR_NAME = "TMAX_daily"
PERCENTILE = 0.9
WINDOW_SIZE = 15
CHUNK_DICT = {"time": 365, "rlat": 50, "rlon": 50}

# === LOAD DATASET ===
print("\n Loading daily max data cube and applying chunking...")
ds = xr.open_dataset(DATASET_PATH).chunk(CHUNK_DICT)
ds = ds.assign_coords(dayofyear=ds.time.dt.dayofyear)

# === INITIALIZE OUTPUT STRUCTURE ===
rlat = ds.rlat
rlon = ds.rlon
doy = np.arange(1, 367)

thresholds_cube = xr.DataArray(
    np.full((366, len(rlat), len(rlon)), np.nan, dtype=np.float32),
    coords={"dayofyear": doy, "rlat": rlat, "rlon": rlon},
    dims=["dayofyear", "rlat", "rlon"],
    name="TMAX_threshold_15d"
)

# === UTILS ===
def format_duration(seconds):
    minutes = int(seconds // 60)
    sec = int(seconds % 60)
    return f"{minutes}m {sec}s"

def print_progress(day, total_days, start_time, end_time):
    percent = 100 * day / total_days
    duration = (end_time - start_time).total_seconds()
    formatted_duration = format_duration(duration)
    print(f"[Progress] Day {day}/{total_days} ({percent:.1f}%) | "
          f"Start: {start_time.strftime('%H:%M:%S')} | End: {end_time.strftime('%H:%M:%S')} | "
          f"Duration: {formatted_duration}")

# === MAIN LOOP ===
print("\n Starting threshold computation for each day of year...")

for d in range(1, 367):
    start_time = datetime.now()

    # Handle day window (±7 days) with wrap-around logic
    offsets = np.arange(-(WINDOW_SIZE // 2), (WINDOW_SIZE // 2) + 1)
    days_window = [(d + offset - 1) % 366 + 1 for offset in offsets]

    ds_window = ds.where(ds.dayofyear.isin(days_window), drop=True)

    # Pixel-wise 90th percentile threshold
    threshold = ds_window[VAR_NAME].quantile(PERCENTILE, dim="time").compute()
    thresholds_cube.loc[dict(dayofyear=d)] = threshold

    end_time = datetime.now()
    print_progress(d, 366, start_time, end_time)

# === SAVE FINAL DATA CUBE ===
thresholds_cube.to_dataset().to_netcdf(OUTPUT_FILE)
print(f"\n Threshold cube saved to: {OUTPUT_FILE}")


 Loading daily max data cube and applying chunking...

 Starting threshold computation for each day of year...
[Progress] Day 1/366 (0.3%) | Start: 08:15:01 | End: 08:15:07 | Duration: 0m 5s
[Progress] Day 2/366 (0.5%) | Start: 08:15:07 | End: 08:15:09 | Duration: 0m 2s
[Progress] Day 3/366 (0.8%) | Start: 08:15:09 | End: 08:15:10 | Duration: 0m 0s
[Progress] Day 4/366 (1.1%) | Start: 08:15:10 | End: 08:15:11 | Duration: 0m 0s
[Progress] Day 5/366 (1.4%) | Start: 08:15:11 | End: 08:15:11 | Duration: 0m 0s
[Progress] Day 6/366 (1.6%) | Start: 08:15:11 | End: 08:15:12 | Duration: 0m 0s
[Progress] Day 7/366 (1.9%) | Start: 08:15:12 | End: 08:15:13 | Duration: 0m 0s
[Progress] Day 8/366 (2.2%) | Start: 08:15:13 | End: 08:15:13 | Duration: 0m 0s
[Progress] Day 9/366 (2.5%) | Start: 08:15:13 | End: 08:15:14 | Duration: 0m 0s
[Progress] Day 10/366 (2.7%) | Start: 08:15:14 | End: 08:15:15 | Duration: 0m 0s
[Progress] Day 11/366 (3.0%) | Start: 08:15:15 | End: 08:15:16 | Duration: 0m 0s
[Progr